In [1]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from datetime import datetime
import json
import sys
from glob import glob
from scipy.stats import gaussian_kde

plt.rcParams['figure.facecolor'] = "white"

In [ ]:
# file where to load data (ERA5 Copernicus)
fname_dataset = "tn_ens_mean_0.1deg_reg_v23.1e.nc"
# minimum population number to add comuni (Italian NUTS-3 levels)
pop_limit = 5e4

In [2]:
# load data from file
f = Dataset(fname_dataset)

# strip times, latitude, and longitude 
times = f["time"][:]
lats = f["latitude"][:]
lons = f["longitude"][:]

# define lat lon range for Italy
idx1 = (35 < lats) & (lats < 47)
idx2 = (6 < lons) & (lons < 20)

lats = lats[idx1]
lons = lons[idx2]

# strip lat/lon range (NOTE: it takes a while!)
tn = f["tn"][:, idx1, idx2]

In [29]:
# prepare geographical data (merge json NUTS-3 levels data)
with open('italy_geo.json') as f:
    geo_ref = json.load(f)

with open('italy_cities.json') as f:
    geo_more = json.load(f)

geo = []
for g in geo_ref:
    if "lat" not in g:
        continue
    if "" in [g["lat"], g["lng"]]:
        continue
    for gm in geo_more:
        if g["istat"] == gm["istat"]:
            if float(gm["num_residenti"]) >= pop_limit:
                geo.append({**g, **gm})
            break
# sort alphabetically
geo = sorted(geo, key=lambda x:x["comune"])
print("found %d city with more than %d population" % (len(geo), pop_limit))

found 142 city with more than 50000 population


In [30]:
def select_data(ymin, ymax, mmm):
    # database starts from 1950
    start_date = datetime.strptime("1/1/1950", "%d/%m/%Y")

    # define time range considering seasons
    t_range = np.zeros_like(times, dtype=bool)
    for y in range(ymin, ymax):
        if mmm == "djf":
            min_date = datetime.strptime("1/12/%d" % (y-1), "%d/%m/%Y")
            max_date = datetime.strptime("28/2/%d" % y, "%d/%m/%Y")
        elif mmm == "mam":
            min_date = datetime.strptime("1/3/%d" % y, "%d/%m/%Y")
            max_date = datetime.strptime("31/5/%d" % y, "%d/%m/%Y")
        elif mmm == "jja":
            min_date = datetime.strptime("1/4/%d" % y, "%d/%m/%Y")
            max_date = datetime.strptime("31/8/%d" % y, "%d/%m/%Y")
        elif mmm == "son":
            min_date = datetime.strptime("1/9/%d" % y, "%d/%m/%Y")
            max_date = datetime.strptime("30/11/%d" % y, "%d/%m/%Y")
        else:
            sys.exit("ERROR: season %s not found!" % mmm)

        # define time range index
        tmin = (min_date - start_date).days
        tmax = (max_date - start_date).days
        
        t_range[tmin:tmax] = True

    # strip time range
    return tn[t_range, ...]

In [37]:
def do_statistics(tn_decade, lats, lons, geo):
    stats = []
    # loop on NUTS-3
    for g in tqdm(geo):
        gname, glat, glon = g["comune"], float(g["lat"]), float(g["lng"])

        # get lat lon corresponding indexes
        i1 = np.argmin(np.abs(lats - glat))
        i2 = np.argmin(np.abs(lons - glon))
        found_ok = False
        # loop around the cells to avoid empty data (i.e. outside the coastline)
        for k in range(0, 10):
            for i in range(-k, k+1):
                for j in range(-k, k+1):
                    tn_data = tn_decade[:, i1+i, i2+j].compressed()
                    if len(tn_data) != 0:
                        if tn_data.min() > -100:
                            found_ok = True
                            break
                if found_ok:
                    break
            if found_ok:
                break
        if not found_ok:
            print("ERROR: %s not found" % gname)
            continue
        
        # do statistical analysis (histogram and KDE)
        bins = np.linspace(tn_data.min(), tn_data.max(), 100)
        h, b = np.histogram(tn_data, bins=bins, density=True)
        b = (b[1:] + b[:-1]) / 2.
        kernel = gaussian_kde(tn_data)
        y = kernel(b) / np.trapz(kernel(b), b)
        
        # get percentiles
        xp = np.percentile(tn_data, [5, 25, 50, 75, 95])

        # this is to plot for debugging
        #print(tn_data.min(), tn_data.max())
        #print(b)
        #print(tn_data)
        #print(np.isfinite(tn_data).all())
        #print(np.isnan(tn_data).any())
        #plt.plot(b, h)
        #plt.plot(b, kernel(b))
        #plt.title(gname)
        #plt.show()
        
        # append results to the data stats structure
        stats.append({"name": gname, "xbin_min": b.min(), "xbin_max": b.max(), "kde": list(y),
                     "xp": list(xp)})
    return stats

In [38]:
# save statistics to file
def save_stats(stats, fname):
    with open(fname, 'w') as f:
        json.dump(stats, f, ensure_ascii=False)

In [39]:
# loop on years and season to produce data (it might take a while)
for ymin in [1950, 1960, 1980, 2000]:
    ymax = ymin + 21
    if ymin == 1950:
        ymax = 2021
    for mmm in ["djf", "mam", "jja", "son"]:
        fname = 'results/%s_%d_%d.json' % (mmm, ymin, ymax)
        tn_decade = select_data(ymin, ymax, mmm)
        stats = do_statistics(tn_decade, lats, lons, geo)
        save_stats(stats, fname)
        print("%s saved" % fname)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 142/142 [00:01<00:00, 71.96it/s]


results/djf_1950_2021.json saved


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 142/142 [00:03<00:00, 45.20it/s]


results/mam_1950_2021.json saved


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 142/142 [00:03<00:00, 35.78it/s]


...

results/mam_2000_2021.json saved


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 150.08it/s]


results/jja_2000_2021.json saved


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 237.52it/s]


results/son_2000_2021.json saved


In [40]:
# load data to write in individual NUTS-3 files, better for web application (smaller files)
data = dict()
for g in glob("results/*.json"):
    with open(g) as f:
        data[g] = json.load(f)

results/mam_1950_2021.json
results/jja_1950_2021.json
results/son_1950_2021.json
results/mam_2000_2021.json
results/djf_2000_2021.json
results/son_1960_1981.json
results/jja_1980_2001.json
results/son_1980_2001.json
results/djf_1960_1981.json
results/son_2000_2021.json
results/mam_1960_1981.json
results/mam_1980_2001.json
results/jja_1960_1981.json
results/djf_1950_2021.json
results/jja_2000_2021.json
results/djf_1980_2001.json


In [41]:
# store into individual files for web application, one for NUTS-3 level
comuni = []
for i, g in enumerate(tqdm(geo)):

    gname = g["comune"]
    comuni.append({"name": gname, "lat": float(g["lat"]), "lon": float(g["lng"])})
    gd = dict()
    for fname, d in data.items():
        k = fname.replace("results/", "").replace(".json", "")
        gd[k] = d[i]
        if d[i]["name"] != gname:
            sys.exit("ERROR!")
    fname = "results_web/data_%06d.json" % i
    with open(fname, 'w') as f:
        json.dump(gd, f, ensure_ascii=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 612.92it/s]


In [42]:
# save NUTS-3 levels to file, for web application
with open("comuni.json", 'w') as f:
    json.dump(comuni, f, ensure_ascii=False)